<a href="https://colab.research.google.com/github/bstungnguyen/AIcolab/blob/main/bolt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cài đặt Node.js và các công cụ cần thiết
!apt-get update
# Thêm NodeSource repository cho Node.js 18
!curl -fsSL https://deb.nodesource.com/setup_18.x | bash -

# Cài Node.js 18 và npm
!apt install -y nodejs
!npm install -g pnpm  # Cài đặt pnpm
# Kiểm tra phiên bản
!node -v
!npm -v
# Clone repository bolt.diy từ GitHub
!git clone https://github.com/stackblitz-labs/bolt.diy.git
%cd bolt.diy

# Cài đặt các phụ thuộc
!pnpm install
# Tải ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin/

# Thêm mã thông báo ngrok (thay YOUR_AUTH_TOKEN bằng token của bạn)
!ngrok authtoken 2mJmtdDIYscHsNAKBbRkasDuOr7_2fXYS3TszsAJk5LXyYkpG
# Chạy một server giả tạm thời trên cổng 5173
import subprocess
import http.server
import socketserver
import threading
import time
# Định nghĩa server giả
PORT = 5173
Handler = http.server.SimpleHTTPRequestHandler
httpd = socketserver.TCPServer(("", PORT), Handler)

# Chạy server trong một luồng riêng
print("Chạy server giả trên cổng 5173 để ngrok tạo tunnel...")
server_thread = threading.Thread(target=httpd.serve_forever)
server_thread.daemon = True
server_thread.start()

# Chạy ngrok để lấy hostname
ngrok_process = subprocess.Popen(['ngrok', 'http', '5173', '--log=stdout'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

# Đọc log từ ngrok để lấy hostname
ngrok_hostname = None
while not ngrok_hostname:
    line = ngrok_process.stdout.readline().decode().strip()
    if "started tunnel" in line and "https://" in line:
        ngrok_url = line.split("url=")[-1]
        ngrok_hostname = ngrok_url.replace("https://", "").replace(":5173", "")
        break

print(f"Ngrok hostname: {ngrok_hostname}")

# Tắt server giả
httpd.shutdown()
httpd.server_close()
print("Đã tắt server giả.")

vite_config_content = f"""
    server: {{
      host: '0.0.0.0',
      port: 5173,
      allowedHosts: [
        'localhost',
        '{ngrok_hostname}',
      ],
    }},
"""
# Đọc nội dung hiện tại của vite.config.ts
with open('/content/bolt.diy/vite.config.ts', 'r') as f:
    current_config = f.read()

# Tìm vị trí bắt đầu của css
css_start = current_config.find('css: {')
if css_start == -1:
    raise Exception("Không tìm thấy css trong vite.config.ts")

# Tìm điểm kết thúc của khối css (sau 3 cặp {})
brace_count = 1
pos = css_start + 6  # Sau "css: {"
while brace_count > 0 and pos < len(current_config):
    if current_config[pos] == '{':
        brace_count += 1
    elif current_config[pos] == '}':
        brace_count -= 1
    pos += 1

css_end = pos + 1  # Điểm ngay sau 3 cặp {}

# Chèn server ngay sau css
new_config = current_config[:css_end] + '\n' + vite_config_content + current_config[css_end:]

# Ghi lại file
with open('/content/bolt.diy/vite.config.ts', 'w') as f:
    f.write(new_config)

# Kiểm tra kết quả
print("Nội dung file vite.config.ts:")
!cat /content/bolt.diy/vite.config.ts

# Chạy bolt.diy
bolt_process = subprocess.Popen('pnpm run dev --host', shell=True, cwd='/content/bolt.diy', stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

# In log từ bolt.diy và hiển thị URL
server_started = False
timeout = time.time() + 60  # Chờ tối đa 60 giây
while time.time() < timeout:
    line = bolt_process.stdout.readline().decode().strip()
    if line:
        print(line)
    if "5173" in line:  # Khi server khởi động
        print(f"Truy cập bolt.diy tại: https://{ngrok_hostname}")
        server_started = True
        break
    if bolt_process.poll() is not None:  # Nếu process dừng
        print("Lỗi: pnpm run dev đã dừng. Kiểm tra log bên dưới.")
        break

# In toàn bộ log nếu có lỗi hoặc tiếp tục in log nếu server chạy
if server_started:
    print("Server đã khởi động. Đang theo dõi log...")
    while True:
        line = bolt_process.stdout.readline().decode().strip()
        if line:
            print(line)
else:
    print("Log lỗi từ pnpm run dev:")
    for line in bolt_process.stdout:
        print(line.decode().strip())
    raise Exception("Server Vite không khởi động thành công.")

In [ ]:
!pip install pyngrok
!wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xzf ngrok-v3-stable-linux-amd64.tgz ngrok
!./ngrok authtoken 2S8EGiMIUPE1Qj3fNmhCJUlxqZO_6G59BkdiDcyKM1oXtwxfA
# 2S8EGiMIUPE1Qj3fNmhCJUlxqZO_6G59BkdiDcyKM1oXtwxfA
# 2mJmtdDIYscHsNAKBbRkasDuOr7_2fXYS3TszsAJk5LXyYkpG
# 2fwvt0DYXdQ0xhfqZsAzFEbegdb_6syaJuEr6vgCxbFNYc996
# 2pEHPTg1WMnNvgdWSgHfv8eEgBf_497kitUfLsjk3aJvtGbL
# 2qn59s6IByfh6XWi6rfvh6fMaWc_3pWuqSsYfgnuChnpTSKC8
# 2r3zl9QpVyvg5JucoEyeMXZW8XN_3iPjJrsKkNNUPfFaZvK64
# 2r40PDAl7yG11rrpFMJ3NekdOLI_WCyRKaKcKP9LdUjiVEnw
# 2r929ru7zBmWKbVmXIqfjVL2zVm_4Zpcz1Ua3JcjeT2AawLa4